# La propagation de labels, ou l'art de juger une image par ses voisins 

Bienvenue dans cette nouvelle expérience ! Dans le chapitre précédent sur le pseudo-labeling, on a vu que notre modèle pouvait devenir un peu trop sûr de lui et finir par tourner en rond, en se confortant dans ses propres erreurs. C'est le fameux **biais de confirmation** ! 

> C'est comme ne parler qu'à des gens qui sont d'accord avec vous : on n'apprend plus rien de nouveau.

**Nos objectifs de super-détective :**
1.  **Recruter un expert** : Charger le modèle qu'on a péniblement entraîné au pseudo-labeling pour qu'il nous aide.
2.  **Cartographier le terrain** : Utiliser cet expert pour extraire l'ADN de chaque image (ses *embeddings*).
3.  **Tisser une toile** : Construire un graphe où chaque image est un nœud, connecté à ses plus proches voisins.
4.  **Laisser la magie opérer** : Regarder les étiquettes de nos 350 images connues se propager à travers la toile pour deviner les autres.
5.  **Comparer les résultats** : Est-ce que cette méthode de 'sagesse des foules' est meilleure que de faire confiance à un seul modèle ? Le suspense est à son comble !

## 1. Préparation du terrain : on reprend (presque) les mêmes !

On commence par importer nos outils et préparer notre jeu de données `DermaMNIST`. On va recréer notre scénario de départ : 350 images étiquetées (50 par classe) et des milliers d'autres qui attendent d'être identifiées.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset
from torchvision import transforms
import torchvision.models as models
import medmnist
from medmnist import INFO, Evaluator
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score

# Pour la reproductibilité, parce qu'on est des gens sérieux
torch.manual_seed(42)
np.random.seed(42)

In [2]:
# Chargement des données
data_flag = 'dermamnist'
info = INFO[data_flag]
n_classes = len(info['label'])
n_channels = info['n_channels']
DataClass = getattr(medmnist, info['python_class'])

# Transformations standard
data_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[.5], std=[.5])])

# On charge le jeu d'entraînement complet et le jeu de test
train_dataset = DataClass(split='train', transform=data_transform, download=True)
test_dataset = DataClass(split='test', transform=data_transform, download=True)

# On recrée notre situation de départ : 50 images par classe étiquetées, et le reste en attente
all_indices = list(range(len(train_dataset)))
labels_array = np.array(train_dataset.labels).flatten()

# Sélectionner 50 images par classe
labeled_indices = []
for c in range(n_classes):
    class_indices = np.where(labels_array == c)[0]
    selected = np.random.choice(class_indices, min(50, len(class_indices)), replace=False)
    labeled_indices.extend(selected)

# Les indices non étiquetés sont le reste
unlabeled_indices = list(set(all_indices) - set(labeled_indices))

print(f'Taille totale du jeu d\'entraînement : {len(train_dataset)} images')
print(f'Données étiquetées (nos indics ) : {len(labeled_indices)} images')
print(f'Données non-étiquetées (les mystères à résoudre ) : {len(unlabeled_indices)} images')

Taille totale du jeu d'entraînement : 7007 images
Données étiquetées (nos indics ) : 350 images
Données non-étiquetées (les mystères à résoudre ) : 6657 images


---

## 2. Recruter notre expert : le modèle du chapitre précédent

Pour que la propagation de labels fonctionne, on a besoin de 'sentir' la similarité entre les images. Utiliser les pixels bruts serait un désastre ! 

On va donc faire appel à un spécialiste : le `SimpleCNN` qu'on a entraîné dans le notebook `P1C3`. Même s'il n'était pas parfait, il a déjà appris à extraire des caractéristiques pertinentes des images de peau. On va lui demander de nous fournir les **embeddings** : une sorte de résumé numérique, ou d'ADN, pour chaque image.

In [9]:
# On définit l'architecture de notre CNN. 
# ATTENTION : Elle doit être IDENTIQUE à celle du modèle sauvegardé !
device = "cpu"
class SimpleCNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(SimpleCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        # Pour correspondre exactement au modèle de P1C3
        self.fc = nn.Linear(7 * 7 * 32, num_classes)

    def forward(self, x, return_features=False):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        if return_features:
            return out  # Retourne les features avant la classification (dimension 1568)
        out = self.fc(out)
        return out

model = SimpleCNN(in_channels=n_channels, num_classes=n_classes)
model_path = 'dermamnist_ssl_model.pth'

try:
    state_dict = torch.load(model_path, map_location="cpu")
    # Les noms de couches correspondent exactement, on charge tout
    model.load_state_dict(state_dict)
    print(f'✅ Modèle chargé depuis : {model_path}')
except FileNotFoundError:
    print(f'🚨 Oups ! Le fichier {model_path} est introuvable.')
    print('Veuillez d\'abord exécuter le notebook P1C3 pour entraîner et sauvegarder le modèle.')
    raise

# On passe le modèle sur le bon appareil et en mode évaluation
model.to("cpu")
model.eval()

✅ Modèle chargé depuis : dermamnist_ssl_model.pth


SimpleCNN(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=7, bias=True)
)

---

## 3. Extraction des 'coordonnées GPS' (embeddings) 

Maintenant que notre expert est prêt, on va le faire passer sur **toutes** les images de notre jeu d'entraînement (étiquetées ou non) pour obtenir leurs fameux embeddings. C'est comme créer une carte d'identité pour chaque image.

In [10]:
def get_embeddings(model, dataset, device):
    """Extrait les embeddings d'un dataset en utilisant un modèle."""
    model.eval()
    embeddings = []
    loader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=2)
    
    with torch.no_grad():
        for images, _ in tqdm(loader, desc='Extraction des embeddings'):
            images = images.to(device)
            feats = model(images, return_features=True)
            embeddings.append(feats.cpu().numpy())
            
    return np.vstack(embeddings)

# On extrait les embeddings en utilisant notre modèle
all_embeddings = get_embeddings(model, train_dataset, "cpu")

print(f'\nExtraction terminée ! On a obtenu {all_embeddings.shape[0]} embeddings de dimension {all_embeddings.shape[1]}.')

Extraction des embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:03<00:00,  8.14it/s]


Extraction terminée ! On a obtenu 7007 embeddings de dimension 1568.


---

## 4. La propagation des rumeurs (de labels) 

C'est le moment que vous attendiez tous ! On va utiliser l'algorithme `LabelSpreading` de scikit-learn.

Comment ça marche ?
1. Il prend tous nos embeddings et construit un graphe de similarité (notre fameuse toile ).
2. On lui donne les 350 étiquettes qu'on connaît. Pour les autres, on met une étiquette spéciale : `-1` (qui veut dire 'Je ne sais pas').
3. L'algorithme va alors 'propager' l'influence des étiquettes connues à leurs voisins, puis aux voisins de leurs voisins, jusqu'à ce que chaque image ait une étiquette probable.

C'est un processus démocratique où chaque image est influencée par sa communauté !

In [11]:
# On prépare le tableau des labels pour l'algorithme
labels_for_spreading = np.full(len(train_dataset), -1, dtype=int)
labels_for_spreading[labeled_indices] = labels_array[labeled_indices]

print(f'Verification : {np.sum(labels_for_spreading != -1)} labels sont connus. Parfait !')

# On instancie le modèle LabelSpreading
label_spreading_model = LabelSpreading(kernel='knn', n_neighbors=10, n_jobs=-1)

print('Propagation des labels en cours... C\'est le moment d\'aller prendre un café ')
label_spreading_model.fit(all_embeddings, labels_for_spreading)
print('Propagation terminée ! Voyons ce qu\'on a trouvé.')

# On récupère les labels prédits pour l'ensemble du dataset
predicted_labels = label_spreading_model.transduction_

# On récupère les probabilités prédites pour l'AUC
predicted_probs = label_spreading_model.predict_proba(all_embeddings)

Verification : 350 labels sont connus. Parfait !
Propagation des labels en cours... C'est le moment d'aller prendre un café 
Propagation terminée ! Voyons ce qu'on a trouvé.


---

## 5. Le verdict : alors, ça a marché ? 

Le modèle a rempli tous les trous et a attribué une étiquette à chaque image. Mais est-ce que ces prédictions sont bonnes?

Pour le savoir, on va comparer les étiquettes prédites pour les données *initialement non-étiquetées* avec leurs vraies étiquettes (qu'on avait cachées). C'est l'heure de vérité !

In [12]:
from torch.utils.data import Dataset

# Créer un dataset personnalisé avec les labels propagés
class PropagatedDataset(Dataset):
    def __init__(self, dataset, labels):
        self.dataset = dataset
        self.labels = labels

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, _ = self.dataset[idx]  # Ignore les labels d'origine, img est déjà un tenseur transformé
        label = self.labels[idx]
        return img, label

# Créer le nouveau dataset avec les labels propagés
train_dataset_propagated = PropagatedDataset(train_dataset, predicted_labels)

# Créer un DataLoader pour l'entraînement
train_loader_propagated = DataLoader(train_dataset_propagated, batch_size=32, shuffle=True, num_workers=2)

In [14]:
def train_and_evaluate(model, train_loader, test_loader, optimizer, criterion, epochs=10):
    """
    Entraîne et évalue un modèle. Retourne (AUC, ACC, F1).
    Si des listes globales metrics_auc/metrics_acc/metrics_f1 existent, y ajoute les scores.
    """
    device = next(model.parameters()).device

    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.squeeze().long().to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Évaluation
    model.eval()
    y_true = torch.tensor([]).to(device)
    y_score_logits = torch.tensor([]).to(device)
    y_score_preds = torch.tensor([]).to(device)
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            y_true = torch.cat((y_true, labels), 0)
            y_score_logits = torch.cat((y_score_logits, outputs), 0)
            preds = torch.argmax(outputs, dim=1)
            y_score_preds = torch.cat((y_score_preds, preds), 0)

    y_true = y_true.squeeze().cpu().numpy()
    y_score_logits = y_score_logits.detach().cpu().numpy()
    y_score_preds = y_score_preds.detach().cpu().numpy()

    evaluator = Evaluator(data_flag, 'test')
    auc, acc = evaluator.evaluate(y_score_logits)
    f1 = f1_score(y_true, y_score_preds, average='macro')

    try:
        metrics_auc.append(auc)
        metrics_acc.append(acc)
        metrics_f1.append(f1)
    except NameError:
        pass

    print(f'AUC: {auc:.3f}, Accuracy: {acc:.3f}, F1: {f1:.3f}')
    return (auc, acc, f1)

In [15]:
# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Entraînement du modèle de base sur images étiquetées avec labels propagés...")
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2)
metrics = train_and_evaluate(model, train_loader_propagated, test_loader, optimizer, criterion)

print('Entraînement terminé !')

Entraînement du modèle de base sur images étiquetées avec labels propagés...
AUC: 0.504, Accuracy: 0.319, F1: 0.316
Entraînement terminé !


## 6. Conclusion et questions pour la suite 

Dans ce run, la propagation de labels sur des embeddings issus d’un petit `SimpleCNN` n’a pas surpassé la boucle de pseudo‑labeling. C’est un résultat fréquent quand les embeddings sont encore « jeunes » et que le graphe n’est pas optimisé. Cela ne remet pas en cause l’intérêt de la méthode : la propagation reste une technique utile pour exploiter la structure globale des données et compléter le pseudo‑labeling.

La propagation de labels est puissante car elle exploite la **structure globale** des données, au lieu de se fier aux prédictions isolées et parfois trop confiantes d'un seul modèle. 

**Mais on peut encore faire mieux ! Voici quelques questions pour ouvrir sur les prochains chapitres :**

1. **La qualité des embeddings** : On a utilisé un petit CNN entraîné sur peu de données. Que se passerait-il si on utilisait un modèle beaucoup plus puissant, comme un **ResNet pré-entraîné sur des millions d'images (ImageNet)**, pour extraire nos embeddings ? La carte serait-elle plus précise ?

2. **Et si on créait de fausses images ?** On manque de données étiquetées. Et si, au lieu de deviner des labels, on demandait à une IA de nous **générer de nouvelles images** de lésions cutanées qui ressemblent aux vraies ? C'est le monde fascinant des **GANs (Generative Adversarial Networks)** que nous explorerons bientôt !

3. **Le meilleur des deux mondes ?** Peut-on combiner le pseudo-labeling et les approches par graphe ? (Indice : oui, et ce sont souvent les méthodes les plus performantes !)